# 🚀 LMFast Quickstart: Train Your First SLM

**Train a production-ready Small Language Model in 5 minutes on free Colab T4!**

## What You'll Learn
- How to install LMFast
- Train a SmolLM-135M model with custom data
- Save and export your model
- Generate text with your fine-tuned model

## Prerequisites
- Google Colab with T4 GPU (free tier works!)
- Basic Python knowledge

**Time to complete:** ~10 minutes (including training)

## 1️⃣ Setup - One-Click Install

In [ ]:
# Install LMFast with all dependencies
!pip install -q lmfast[all]

# Verify GPU is available
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Import LMFast
import lmfast

# Setup Colab environment (optimizes for T4)
lmfast.setup_colab_env()

print(f"LMFast version: {lmfast.__version__}")

## 2️⃣ Prepare Your Data

LMFast accepts data in multiple formats:
- HuggingFace datasets
- JSON files
- CSV files
- Python lists

For this quickstart, we'll use the Alpaca dataset (instruction-following).

In [ ]:
from datasets import load_dataset

# Load a small subset of Alpaca for quick training
dataset = load_dataset("yahma/alpaca-cleaned", split="train[:1000]")

print(f"Dataset size: {len(dataset)} examples")
print(f"\nExample entry:")
print(dataset[0])

## 3️⃣ Configure Your Model

LMFast uses Pydantic configs for validation. All defaults are optimized for Colab T4.

In [ ]:
from lmfast import SLMConfig, TrainingConfig

# Model configuration
model_config = SLMConfig(
    model_name="HuggingFaceTB/SmolLM-135M",  # Fastest model for learning
    max_seq_length=512,
    load_in_4bit=True,  # QLoRA for memory efficiency
)

# Training configuration
training_config = TrainingConfig(
    output_dir="./my_first_slm",
    max_steps=200,  # ~5 min on T4
    batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    lora_r=16,
    lora_alpha=32,
)

# Check T4 compatibility
print(f"T4 Compatible: {model_config.is_colab_t4_compatible()}")
print(f"Effective Batch Size: {training_config.effective_batch_size}")
print(f"Estimated Training Time: {training_config.estimated_training_time_minutes:.0f} min")

## 4️⃣ Train Your Model! 🎯

This is where the magic happens. LMFast handles:
- Data formatting and tokenization
- LoRA adapter injection
- GPU memory optimization
- Progress tracking

In [ ]:
from lmfast import SLMTrainer

# Create trainer
trainer = SLMTrainer(
    model_config=model_config,
    training_config=training_config,
    use_unsloth=True  # 2-5x faster training!
)

# Train!
print("🚀 Starting training...")
trainer.train(dataset)
print("✅ Training complete!")

## 5️⃣ Save Your Model

In [ ]:
# Save the trained model
trainer.save("./my_first_slm")
print("✅ Model saved to ./my_first_slm")

# List saved files
import os
for f in os.listdir("./my_first_slm"):
    size_mb = os.path.getsize(f"./my_first_slm/{f}") / 1e6
    print(f"  {f}: {size_mb:.1f} MB")

## 6️⃣ Test Your Model

In [ ]:
from lmfast.inference import SLMServer

# Create inference server
server = SLMServer("./my_first_slm")

# Test generation
prompts = [
    "What is machine learning?",
    "Write a haiku about programming.",
    "Explain the benefits of small language models."
]

for prompt in prompts:
    print(f"\n📝 Prompt: {prompt}")
    response = server.generate(
        prompt,
        max_new_tokens=100,
        temperature=0.7
    )
    print(f"🤖 Response: {response}")
    print("-" * 50)

## 7️⃣ (Optional) Export to GGUF for Local Use

Export your model for use with llama.cpp, Ollama, or other local inference engines.

In [ ]:
from lmfast.inference.quantization import export_gguf

# Export to GGUF (requires llama.cpp - may not work in Colab)
try:
    export_gguf(
        "./my_first_slm",
        "./my_first_slm.gguf",
        quantization="q4_k_m"
    )
    print("✅ GGUF export complete!")
except Exception as e:
    print(f"⚠️ GGUF export requires llama.cpp: {e}")
    print("💡 Tip: For GGUF export, use a local machine with llama.cpp installed.")

## 🎉 Congratulations!

You've just trained your first Small Language Model with LMFast!

### Next Steps

1. **Custom Data**: See `02_custom_dataset.ipynb` for data preparation
2. **Better Performance**: See `05_knowledge_distillation.ipynb` for distillation
3. **Build Agents**: See `09_basic_agents.ipynb` for tool-using agents
4. **Production Deploy**: See `04_inference_server.ipynb` for serving

### Key Takeaways

- ✅ Train SLMs on free Colab T4 in minutes
- ✅ LMFast handles optimization automatically
- ✅ Export to multiple formats (GGUF, INT4, etc.)
- ✅ Simple API: just 5 lines of code to train!

### Resources

- 📚 [Documentation](https://lmfast.readthedocs.io)
- 💬 [Discord Community](https://discord.gg/lmfast)
- 🐛 [GitHub Issues](https://github.com/2796gaurav/LMFast/issues)

In [ ]:
# One-line training alternative!
# If you want the simplest possible experience:

# import lmfast
# lmfast.train(
#     model="HuggingFaceTB/SmolLM-135M",
#     dataset="yahma/alpaca-cleaned",
#     output_dir="./quick_model",
#     max_steps=200
# )